In [217]:
import pandas as pd

In [218]:
train = pd.read_csv('train_V2.csv')

In [219]:
test = pd.read_csv('test_V2.csv')

In [220]:
train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [221]:
test.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
0,9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,0,0,51.46,0,0,0,73,...,1500,0,0.0,0,0.0,0,0,588.0,1,0
1,639bd0dcd7bda8,430933124148dd,42a9a0b906c928,0,4,179.10,0,0,2,11,...,1503,2,4669.0,0,0.0,0,0,2017.0,6,0
2,63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,1,0,23.40,0,0,4,49,...,1565,0,0.0,0,0.0,0,0,787.8,4,0
3,cf5b81422591d1,b7497dbdc77f4a,1b9a94f1af67f1,0,0,65.52,0,0,0,54,...,1465,0,0.0,0,0.0,0,0,1812.0,3,0
4,ee6a295187ba21,6604ce20a1d230,40754a93016066,0,4,330.20,1,2,1,7,...,1480,1,0.0,0,0.0,0,0,2963.0,4,0


In [222]:
train = train.drop(train[train['walkDistance']==0][train['kills']>0].index, axis = 0)

C:\Users\shjas\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [223]:
def rstr(df, pred=None):
    obs = df.shape[0]
    types = df.dtypes
    counts = df.apply(lambda x: x.count())
    uniques = df.apply(lambda x: [x.unique()])
    nulls = df.apply(lambda x: x.isnull().sum())
    distincts = df.apply(lambda x: x.unique().shape[0])
    missing_ration = (df.isnull().sum()/obs) *100
    skewness = df.skew()
    kurtosis = df.kurt()
    print('Data shape: ', df.shape)
    
    if pred is None:
        cols = ['types', 'counts', 'distincts', 'nulls', 'missing ration', 'uniques', 'skewness', 'kurtosis', 'corr']
        str = pd.concat([types, counts, distincts, nulls, missing_ration, uniques, skewness, kurtosis], axis=1)
    else:
        corr = df.corr()[pred]
        str =pd.concat([types, counts, distincts, nulls, missing_ration, uniques, skewness, kurtosis, corr], axis=1, sort=False)
        corr_col = 'corr ' + pred
        cols = ['types', 'counts', 'distincts', 'nulls', 'missing ration', 'uniques', 'skewness', 'kurtosis',  corr_col]
    str.columns = cols
    dtypes = str.types.value_counts()
    print('___________________________\nData types:\n',str.types.value_counts())
    print('___________________________')
    return str

# walkDistance, killplace, boosts, weaponsAcquired -> 매우 중요
## damageDealt, heals, kills, longest kills -> 중요

In [224]:
# pd.set_option('display.max_rows', None)
# details = rstr(train, 'winPlacePerc')
# display(details.sort_values(by='corr winPlacePerc', ascending=False))

In [225]:
train["group_size"] = train.groupby(["matchId"])['Id'].transform('count')
train["killPlace"] = train["killPlace"]/train["group_size"]
train.drop('group_size', axis = 1, inplace = True)

In [226]:
train["distanceSum"] = 0.81*train['walkDistance'] + 0.34*train['rideDistance'] + 0.14*train['swimDistance']
train.drop(['walkDistance', 'rideDistance','swimDistance'], axis=1, inplace = True)

In [227]:
train["item"] = 0.63*train['boosts'] + 0.42*train['heals']
train.drop(['boosts', 'heals'], axis=1, inplace=True)

In [228]:
train["killplace"] = -1*train["killPlace"]

In [229]:
etc = train[['numGroups', 'Id', 'groupId','matchId', 'matchType']]
train.drop(['vehicleDestroys','numGroups', 'Id', 'groupId','matchId', 'matchType','rankPoints','killPoints','winPoints','matchDuration',
           'roadKills','teamKills','maxPlace','killPlace', 'kills'], axis=1, inplace=True)

## 선정한 중요 변수들 사이의 상관관계 분석

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(13,13))
hm = sns.heatmap(train[['walkDistance', 'killPlace','boosts','weaponsAcquired','damageDealt','heals','kills','longestKill']].corr(), 
                 annot=True, annot_kws={'size':13})
bottom, top = hm.get_ylim()
hm.set_ylim(bottom+0.5, top-0.5)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(25,25))
hm = sns.heatmap(train.corr(), 
                 annot=True, annot_kws={'size':13})
bottom, top = hm.get_ylim()
hm.set_ylim(bottom+0.5, top-0.5)
plt.tight_layout()
plt.show()

## walkDistance == 0이면서 kills > 0인 애들 -> 핵 의심

## 중요변수만으로 선형회귀 돌려보기

In [230]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [231]:
target = train['winPlacePerc']
target = target.fillna(target.mean())
train.drop('winPlacePerc', axis=1, inplace=True)

In [232]:
train.isnull().sum()

assists            0
damageDealt        0
DBNOs              0
headshotKills      0
killStreaks        0
longestKill        0
revives            0
weaponsAcquired    0
distanceSum        0
item               0
killplace          0
dtype: int64

In [233]:
x_train, x_test, y_train, y_test = train_test_split(train, target, train_size = 0.7, test_size = 0.3, random_state = 25)

In [234]:
lr = LinearRegression()
ridge = Ridge(alpha = 0.5)
lasso = Lasso(alpha = 0.5)
elanet = ElasticNet(alpha = 0.5, l1_ratio = 0.5)
Xgb = XGBRegressor(max_depth=10, n_jobs=-1, n_estimators=500, tree_method='gpu_hist', gpu_id=0)
cat = CatBoostRegressor(max_depth=10, n_estimators=500, task_type="GPU", verbose=2)

In [235]:
lr_model = lr.fit(x_train, y_train)
ridge_model = ridge.fit(x_train, y_train)
lasso_model = lasso.fit(x_train, y_train)
elanet_model = elanet.fit(x_train, y_train)
xgb_model = Xgb.fit(x_train, y_train)
cat_model = cat.fit(x_train, y_train)

[20:52:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Learning rate set to 0.395445


0:	learn: 0.2075065	total: 60.9ms	remaining: 30.4s
2:	learn: 0.1246926	total: 184ms	remaining: 30.5s
4:	learn: 0.1044676	total: 308ms	remaining: 30.5s
6:	learn: 0.0996777	total: 426ms	remaining: 30s
8:	learn: 0.0979317	total: 540ms	remaining: 29.5s
10:	learn: 0.0968506	total: 660ms	remaining: 29.3s
12:	learn: 0.0963571	total: 780ms	remaining: 29.2s
14:	learn: 0.0959610	total: 893ms	remaining: 28.9s
16:	learn: 0.0956657	total: 1.01s	remaining: 28.6s
18:	learn: 0.0953623	total: 1.12s	remaining: 28.4s
20:	learn: 0.0951238	total: 1.24s	remaining: 28.2s
22:	learn: 0.0949780	total: 1.35s	remaining: 28s
24:	learn: 0.0948253	total: 1.47s	remaining: 27.9s
26:	learn: 0.0946317	total: 1.58s	remaining: 27.6s
28:	learn: 0.0944720	total: 1.69s	remaining: 27.5s
30:	learn: 0.0943353	total: 1.81s	remaining: 27.4s
32:	learn: 0.0942316	total: 1.92s	remaining: 27.2s
34:	learn: 0.0941533	total: 2.03s	remaining: 27s
36:	learn: 0.0940739	total: 2.17s	remaining: 27.1s
38:	learn: 0.0940117	total: 2.28s	remaini

In [236]:
print("훈련 스코어(lr)     : %.4f" % lr_model.score(x_train, y_train))
print("훈련 스코어(ridge)  : %.4f" % ridge_model.score(x_train, y_train))
print("훈련 스코어(lasso)  : %.4f" % lasso_model.score(x_train, y_train))
print("훈련 스코어(elanet) : %.4f" % elanet_model.score(x_train, y_train))
print("훈련 스코어(xgb)    : %.4f" % xgb_model.score(x_train, y_train))
print("훈련 스코어(cat)    : %.4f" % cat_model.score(x_train, y_train))

훈련 스코어(lr)     : 0.8016
훈련 스코어(ridge)  : 0.8016
훈련 스코어(lasso)  : 0.6375
훈련 스코어(elanet) : 0.6380
훈련 스코어(xgb)    : 0.9198
훈련 스코어(cat)    : 0.9137


In [237]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [238]:
print("예측 스코어(lr)     : %.4f" % r2_score(y_test, lr_model.predict(x_test))) 
print("예측 스코어(ridge)  : %.4f" % r2_score(y_test, ridge_model.predict(x_test)))
print("예측 스코어(lasso)  : %.4f" % r2_score(y_test, lasso_model.predict(x_test))) 
print("예측 스코어(elanet) : %.4f" % r2_score(y_test, elanet_model.predict(x_test))) 
print("예측 스코어(xgb)    : %.4f" % r2_score(y_test, xgb_model.predict(x_test)))
print("예측 스코어(cat)    : %.4f" % r2_score(y_test, cat_model.predict(x_test)))

예측 스코어(lr)     : 0.8021
예측 스코어(ridge)  : 0.8021
예측 스코어(lasso)  : 0.6381
예측 스코어(elanet) : 0.6386
예측 스코어(xgb)    : 0.9084
예측 스코어(cat)    : 0.9083


In [239]:
print("예측 mae(lr)     : %.4f" % mean_absolute_error(y_test, lr_model.predict(x_test))) 
print("예측 mae(ridge)  : %.4f" % mean_absolute_error(y_test, ridge_model.predict(x_test))) 
print("예측 mae(lasso)  : %.4f" % mean_absolute_error(y_test, lasso_model.predict(x_test))) 
print("예측 mae(elanet) : %.4f" % mean_absolute_error(y_test, elanet_model.predict(x_test)))
print("예측 mae(xgb)    : %.4f" % mean_absolute_error(y_test, xgb_model.predict(x_test)))
print("예측 mae(cat)    : %.4f" % mean_absolute_error(y_test, cat_model.predict(x_test)))

예측 mae(lr)     : 0.1002
예측 mae(ridge)  : 0.1002
예측 mae(lasso)  : 0.1498
예측 mae(elanet) : 0.1497
예측 mae(xgb)    : 0.0656
예측 mae(cat)    : 0.0657


## matchType별 그룹화, 듀오 이상인 경우는 게임 플레이 관련 데이터 전부 팀별 평균으로 대체

In [ ]:
matchtype = train.matchType.unique()

In [ ]:
matchtype

## squad-### : 4
## duo-### : 2
## solo-### : 1
## crash###, flare### : 0

In [ ]:
train['matchType'] = train['matchType'].map({'squad-fpp':4, 'squad':4, 'normal-squad-fpp':4, 'normal-squad':4,
                        'duo':2, 'duo-fpp':2, 'normal-duo-fpp':2, 'normal-duo':2,
                        'solo-fpp':1, 'solo':1, 'normal-solo-fpp':1, 'normal-solo':1,
                        'crashfpp':0, 'flaretpp':0, 'flarefpp':0, 'crashtpp':0})